In [1]:
import pandas as pd
from spacy.cli import download
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import spacy

In [ ]:
train_data = pd.read_csv('augmented data/augmented_data_with_gpt2.csv')
val_data = pd.read_csv('val_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'augmented data/augmented_data_with_gpt2.csv'

In [ ]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657.0,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet fatal concurrent map writes while look...,False
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620.0,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,failed verify pod status checkpoint checksum b...,False
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259.0,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore flag flag removed moved removed depend...,False
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175.0,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,multiple which jobs are which tests are node c...,False
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182.0,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash kube after kube manager started no looks...,False


In [ ]:
# check for null values
train_data.isnull().sum()


comments_url    274
id                0
title             0
body              0
issue_url       274
pr_url          274
labels            0
pr_number       274
filename        274
status          274
additions       274
deletions       274
changes         274
all_comments    301
all_text          0
is_augmented      0
dtype: int64

In [ ]:
val_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text
0,https://api.github.com/repos/kubernetes/kubern...,452179309,Some CRDs that duplicate built-in group/versio...,**What happened**:\r\n\r\nCreated a CRD:\r\n``...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/custom-resources']",82035,['staging/src/k8s.io/apiextensions-apiserver/p...,"['modified', 'modified', 'modified', 'added']","[1, 20, 9, 380]","[0, 1, 0, 0]","[1, 21, 9, 380]",@connor1989 /sub Is anyone working on this? I'...,some crds that duplicate reap etcd data deleti...
1,https://api.github.com/repos/kubernetes/kubern...,452175013,Some CRDs that duplicate built-in group/versio...,**What happened**:\r\nCreated a CRD with a gro...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/custom-resources', 'area/d...",81436,['staging/src/k8s.io/apiextensions-apiserver/g...,"['modified', 'modified', 'modified', 'modified...","[1, 2, 1, 14, 5, 209]","[0, 0, 0, 5, 1, 0]","[1, 2, 1, 19, 6, 209]",/cc @cheftako haven't looked into the code ye...,some crds that duplicate show up openapi creat...
2,https://api.github.com/repos/kubernetes/kubern...,451615547,Azure Disk detach backoff cancelled due to err...,**What happened**:\r\n\r\n#78298 made sure tha...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],78700,['staging/src/k8s.io/legacy-cloud-providers/az...,"['modified', 'modified']","[2, 2]","[2, 2]","[4, 4]",/assign,azure disk detach backoff cancelled due error ...
3,https://api.github.com/repos/kubernetes/kubern...,451156427,"""--kube-reserved-cgroup"" kubelet option cannot...",start kubelet with below config:\r\n```\r\n# r...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],78793,['pkg/kubelet/cm/cgroup_manager_linux_test.go'...,"['modified', 'modified']","[22, 2]","[0, 2]","[22, 4]",/area kubelet\r\n/sig node\r\n @derekwaynecarr...,kubelet option cannot find cgroup start kubele...
4,https://api.github.com/repos/kubernetes/kubern...,451147670,"Windows kubelet won't start ""Failed to create ...",**What happened**:\r\n\r\nI replaced the kubel...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/test', 'area/kubelet']",78704,"['pkg/kubelet/kubelet.go', 'pkg/kubelet/util/u...","['modified', 'modified', 'modified', 'modified...","[6, 2, 2, 3, 4]","[1, 2, 2, 7, 1]","[7, 4, 4, 10, 5]",cc @adelina-t \r\n/priority critical-urgent /m...,windows kubelet start create listener podresou...


Check the label distribution

In [ ]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in train_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Training Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Training Data):
1. area/test: 309
2. area/kubelet: 304
3. area/apiserver: 196
4. area/cloudprovider: 171
5. area/kubectl: 133
6. area/dependency: 80
7. area/code-generation: 65
8. area/provider/azure: 64
9. area/ipvs: 39
10. area/kubeadm: 33
11. area/provider/gcp: 32
12. area/api: 28
13. area/e2e-test-framework: 28
14. area/kube-proxy: 28
15. area/release-eng: 28
16. area/conformance: 28
17. area/batch: 28
18. area/deflake: 28
19. area/network-policy: 28
20. area/client-libraries: 28
21. area/code-organization: 28
22. area/security: 28
23. area/etcd: 28
24. area/custom-resources: 28
25. area/provider/aws: 28


In [ ]:
# Flatten the list of labels if it's a list of lists
flattened_labels = []
for label_list in val_data["labels"]:
    # Convert string representation of list to actual list
    if isinstance(label_list, str):
        # Remove brackets and split by comma
        clean_label_list = label_list.strip('[]').replace("'", "").split(', ')
    else:
        clean_label_list = label_list
    
    # Add each label to the flattened list
    flattened_labels.extend(clean_label_list)

# Print labels distribution count
label_counts = Counter(flattened_labels)
print("Label Distribution Count (Validation Data):")

for i, (label, count) in enumerate(label_counts.most_common()):
    print(f"{i+1}. {label}: {count}")

Label Distribution Count (Validation Data):
1. area/kubelet: 79
2. area/apiserver: 45
3. area/cloudprovider: 42
4. area/test: 32
5. area/kubectl: 29
6. area/kubeadm: 18
7. area/ipvs: 13
8. area/dependency: 12
9. area/custom-resources: 9
10. area/code-generation: 6
11. area/provider/vmware: 6
12. area/kube-proxy: 6
13. area/security: 6
14. area/provider/openstack: 5
15. area/conformance: 5
16. area/provider/azure: 3
17. area/e2e-test-framework: 2
18. area/provider/gcp: 1


Stopwords Removal

In [3]:
# remove stopwords using spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.text for word in nlp(x) if not word.is_stop]))

NameError: name 'train_data' is not defined

In [119]:
# check null values in all_text column
# Display null values for all columns
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Data Lemmatization

In [120]:
# lematize the text using spaCy
train_data['all_text'] = train_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))
val_data['all_text'] = val_data['all_text'].apply(lambda x: ' '.join([word.lemma_ for word in nlp(x)]))

In [121]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


Add filename to the text

In [122]:
# add filename to the data
# Only append filename if it exists (not null/None)
train_data['all_text'] = train_data.apply(lambda row: row['all_text'] + ' ' + row['filename'] if pd.notna(row['filename']) else row['all_text'], axis=1)
val_data['all_text'] = val_data.apply(lambda row: row['all_text'] + ' ' + row['filename'] if pd.notna(row['filename']) else row['all_text'], axis=1)

In [123]:
# check all_text column for null values
print(f"Null all_text column : {train_data['all_text'].isnull().sum()}")

Null all_text column : 0


# Export the Data

In [124]:
train_data.head()

,comments_url,id,title,body,issue_url,pr_url,labels,pr_number,filename,status,additions,deletions,changes,all_comments,all_text,is_augmented
0,https://api.github.com/repos/kubernetes/kubern...,2639668210,kubelet crash: fatal error: concurrent map writes,### What happened?\n\nWhile looking into three...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],128657.0,"['pkg/kubelet/cm/container_manager_linux.go', ...","['modified', 'modified', 'modified', 'modified']","[3, 4, 18, 75]","[3, 3, 6, 0]","[6, 7, 24, 75]",/sig node Thought to search for similar errors...,kubelet fatal concurrent map write look fail e...,False
1,https://api.github.com/repos/kubernetes/kubern...,2617512099,[FG:InPlacePodVerticalScaling] failed to verif...,### What happened?\n\nOne line bug description...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/kubelet'],126620.0,"['pkg/kubelet/status/state/checkpoint.go', 'pk...","['modified', 'modified', 'added']","[42, 23, 166]","[26, 17, 0]","[68, 40, 166]",This issue is currently awaiting triage.\n\nIf...,fail verify pod status checkpoint checksum dif...,False
2,https://api.github.com/repos/kubernetes/kubern...,2604613192,Restore build-tag flag for code-generator,### What happened?\n\nThe `build-tag` flag is ...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/code-generation'],128259.0,['staging/src/k8s.io/code-generator/cmd/conver...,"['modified', 'modified', 'modified', 'modified...","[12, 1, 1, 12, 1, 1]","[2, 1, 1, 1, 1, 1]","[14, 2, 2, 13, 2, 2]",@p0lyn0mial FYI /sig api-machinery /triage acc...,restore flag flag remove move remove dependenc...,False
3,https://api.github.com/repos/kubernetes/kubern...,2596132738,[Failing Tests] ci-crio-cgroupv1-node-e2e-conf...,### Which jobs are failing?\n* master-blocking...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,"['area/kubeadm', 'area/dependency']",128175.0,"['go.mod', 'go.sum', 'hack/unwanted-dependenci...","['modified', 'modified', 'modified', 'modified...","[1, 3, 1, 21, 9, 9, 4, 0, 10, 2, 2, 5, 8, 1]","[1, 2, 0, 98, 8, 7, 6, 62, 11, 1, 4, 8, 11, 1]","[2, 5, 1, 119, 17, 16, 10, 62, 21, 3, 6, 13, 1...",@drewhagen: The provided milestone is not vali...,multiple job test node conformance test testgr...,False
4,https://api.github.com/repos/kubernetes/kubern...,2591358936,Crash on kube manager's service-lb-controller ...,### What happened?\n\nIf kube manager is start...,https://github.com/kubernetes/kubernetes/issue...,https://github.com/kubernetes/kubernetes/pull/...,['area/cloudprovider'],128182.0,['cmd/kube-controller-manager/app/controllerma...,"['modified', 'modified', 'modified']","[26, 12, 5]","[0, 1, 4]","[26, 13, 9]",This issue is currently awaiting triage.\n\nIf...,crash kube kube manager start look like instan...,False


In [125]:
# count nan values in the train_data
train_data.isna().sum()

comments_url    274
id                0
title             0
body              0
issue_url       274
pr_url          274
labels            0
pr_number       274
filename        274
status          274
additions       274
deletions       274
changes         274
all_comments    301
all_text          0
is_augmented      0
dtype: int64

In [127]:
# # export the train_data and val_data to csv
train_data.to_csv('preprocessed data/preprocessed_train_data.csv', index=False)
val_data.to_csv('preprocessed data/preprocessed_val_data.csv', index=False)